## Preparation

In [1]:
import sys
sys.path

['D:\\bio\\jupyter_code\\seqFish-SpatialMouseAtlas',
 'D:\\anaconda\\envs\\STAligner\\python310.zip',
 'D:\\anaconda\\envs\\STAligner\\DLLs',
 'D:\\anaconda\\envs\\STAligner\\lib',
 'D:\\anaconda\\envs\\STAligner',
 '',
 'C:\\Users\\23147\\AppData\\Roaming\\Python\\Python310\\site-packages',
 'D:\\anaconda\\envs\\STAligner\\lib\\site-packages',
 'D:\\anaconda\\envs\\STAligner\\lib\\site-packages\\staligner-1.0.0-py3.10.egg',
 'D:\\anaconda\\envs\\STAligner\\lib\\site-packages\\win32',
 'D:\\anaconda\\envs\\STAligner\\lib\\site-packages\\win32\\lib',
 'D:\\anaconda\\envs\\STAligner\\lib\\site-packages\\Pythonwin']

In [2]:
#-*- coding : utf-8-*-
# coding:unicode_escape
import warnings
warnings.filterwarnings("ignore")

# import ST_utils
# import train_STAligner
import STAligner

# the location of R (used for the mclust clustering)
import os
os.environ['R_HOME'] = "D:\\anaconda\envs\STAligner\Lib\R"
os.environ['R_USER'] = "D:\\anaconda\envs\STAligner\Lib\site-packages\rpy2"

In [3]:
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri

In [4]:
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.sparse as sp
import scipy.linalg

import torch
used_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(used_device)

datasets = ["embryo1-2","embryo1-5","embryo2-2","embryo2-5","embryo3-2","embryo3-5"]
experiment_name = 'allEmbryo'
input_dir = 'G:/dataset/seqFish-SpatialMouseAtlas/output/python/'  # Replace it with your file path
output_dir = 'G:/dataset/seqFish-SpatialMouseAtlas/output/SPACEL/'  # Replace it with your file path

# num_cluster = 24

cuda:0


In [5]:
# import pandas as pd
# section_ids = ['Anterior1','Anterior2','Posterior1','Posterior2']
# for dataset in section_ids:
#     adata_label = pd.read_csv(input_dir + dataset + '/truth.csv', index_col=0)
#     types = adata_label['celltype'].unique().tolist()
#     print(len(types))

## Load Data

In [6]:
from scipy import sparse

Batch_list = []
adj_list = []
print(datasets)

for section_id in datasets:
    print(section_id)
    # input_path = input_dir + section_id
    # adata = sc.read_visium(path=input_path, count_file='filtered_feature_bc_matrix.h5', load_images=True)
    # adata.var_names_make_unique(join="++")

    matrix = pd.read_csv(input_dir + section_id + "/matrix.csv", index_col=False)
    feature = pd.read_csv(input_dir + section_id + "/features.csv", index_col=0)
    feature.index.name = 'gene'
    barcode = pd.read_csv(input_dir + section_id + "/barcodes.csv", index_col=0)
    barcode.index.name = 'spot'
    adata = ad.AnnData(X=matrix.to_numpy(dtype='float64').T, obs=barcode, var=feature)
    adata = ad.AnnData(X=sparse.csr_matrix(matrix.to_numpy(dtype='float32').T), obs=barcode, var=feature)
    
    positions = pd.read_csv(input_dir + section_id + "/tissue_positions_list.csv", index_col=0, header=None)
    positions.drop(positions.columns[[0, 3, 4]], axis=1, inplace=True)
    positions.columns = ['x', 'y']
    adata.obsm['spatial'] = positions.values
    
    adata_label = pd.read_csv(input_dir + section_id + '/truth.csv', index_col=0)
    # adata_label.index = [str.replace(str.replace(x, '.', '-'), adata_label['batch'][0] + '-', '') for x in adata_label.index]
    adata.obs['batch'] = adata_label['batch']
    # adata.obs['annotation'] = adata_label['celltype_mapped_refined']
    adata.obs['Ground Truth'] = adata_label.loc[adata.obs_names, 'celltype_mapped_refined'].astype('category')

    # # read the annotation
    # Ann_df = pd.read_csv(os.path.join(input_path, 'truth.csv'), index_col=0)
    # Ann_df.index = [str.replace(str.replace(x, '.', '-'), Ann_df['batch'][0] + '-', '') for x in Ann_df.index]
    # # Ann_df['ground.truth'] = Ann_df['celltype'].replace('na', 'unknown')
    # adata.obs['Ground Truth'] = Ann_df.loc[adata.obs_names, 'celltype'].astype('category')

    # make spot name unique
    adata.obs_names = [x + '_' + section_id for x in adata.obs_names]

    # Constructing the spatial network
    if(section_id == "embryo3-2" or section_id == "embryo3-5"):
        STAligner.Cal_Spatial_Net(adata, rad_cutoff=4)
    else:
        STAligner.Cal_Spatial_Net(adata, rad_cutoff=2)
    # the spatial network are saved in adata.uns[‘adj’]
    # STAligner.Stats_Spatial_Net(adata) # plot the number of spatial neighbors

    # Normalization
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=5000)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata = adata[:, adata.var['highly_variable']]

    adj_list.append(adata.uns['adj'])
    Batch_list.append(adata)

['embryo1-2', 'embryo1-5', 'embryo2-2', 'embryo2-5', 'embryo3-2', 'embryo3-5']
embryo1-2
------Calculating spatial graph...
The graph contains 70134 edges, 11026 cells.
6.3608 neighbors per cell on average.
embryo1-5
------Calculating spatial graph...
The graph contains 55332 edges, 8425 cells.
6.5676 neighbors per cell on average.
embryo2-2
------Calculating spatial graph...
The graph contains 48042 edges, 8011 cells.
5.9970 neighbors per cell on average.
embryo2-5
------Calculating spatial graph...
The graph contains 43700 edges, 6880 cells.
6.3517 neighbors per cell on average.
embryo3-2
------Calculating spatial graph...
The graph contains 64558 edges, 12541 cells.
5.1478 neighbors per cell on average.
embryo3-5
------Calculating spatial graph...
The graph contains 58284 edges, 10653 cells.
5.4711 neighbors per cell on average.


## Concat the scanpy objects for multiple slices

In [7]:
adata_concat = ad.concat(Batch_list, label="slice_name", keys=datasets)
adata_concat.obs['Ground Truth'] = adata_concat.obs['Ground Truth'].astype('category')
adata_concat.obs["batch_name"] = adata_concat.obs["slice_name"].astype('category')
print('adata_concat.shape: ', adata_concat.shape)

adata_concat.shape:  (57536, 351)


## Concat the spatial network for multiple slices

In [8]:
adj_concat = np.asarray(adj_list[0].todense())
for batch_id in range(1,len(datasets)):
    adj_concat = scipy.linalg.block_diag(adj_concat, np.asarray(adj_list[batch_id].todense()))
adata_concat.uns['edgeList'] = np.nonzero(adj_concat)

MemoryError: Unable to allocate 16.4 GiB for an array with shape (46883, 46883) and data type float64

## Running STAligner

In [ ]:
%%time
adata_concat = STAligner.train_STAligner(adata_concat, verbose=True, knn_neigh = 50, device=used_device)

# save embedding

In [ ]:
# save embedding
np.savetxt(out_dir + expriment_name + '_cluster_' + str(num_cluster) + '_STAligner.csv', adata_concat.obsm['STAligner'], delimiter=",")
np.savetxt(out_dir + expriment_name + '_cluster_' + str(num_cluster) + '_STAGATE.csv', adata_concat.obsm['STAGATE'], delimiter=",")

## Clustering

In [ ]:
adata_concat

In [ ]:
# import seaborn as sns
# slice_colors = sns.color_palette(n_colors = 6).as_hex()
df = pd.DataFrame()
for i in range(len(adata_list)):
    df = pd.concat([df,adata_list[i].obs],axis=0)

series = df.iloc[:, 1]
series = series.astype("category")

celltype_num = len(series.value_counts().index)
celltypes = series.value_counts().index.tolist()

import seaborn as sns
colors = sns.color_palette(n_colors = celltype_num).as_hex()

In [ ]:
num_cluster = celltype_num

In [ ]:
from STAligner import ST_utils
ST_utils.mclust_R(adata_concat, num_cluster=num_cluster, used_obsm='STAligner')
adata_concat = adata_concat[adata_concat.obs['Ground Truth']!='unknown']

In [ ]:
from sklearn.metrics import adjusted_rand_score as ari_score
print('mclust, ARI = %01.3f' % ari_score(adata_concat.obs['Ground Truth'], adata_concat.obs['mclust']))

## Visualization

In [ ]:
sc.pp.neighbors(adata_concat, use_rep='STAligner', random_state=666)
sc.tl.umap(adata_concat, random_state=666)

# section_color = ['#f8766d', '#7cae00', '#00bfc4', '#c77cff']
section_color = sns.color_palette(n_colors = len(datasets)).as_hex()
section_color_dict = dict(zip(section_ids, section_color))
adata_concat.uns['batch_name_colors'] = [section_color_dict[x] for x in adata_concat.obs.batch_name.cat.categories]
adata_concat.obs['mclust'] = pd.Series(ST_utils.match_cluster_labels(adata_concat.obs['Ground Truth'], adata_concat.obs['mclust'].values),
                                         index=adata_concat.obs.index, dtype='category')

import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams["figure.figsize"] = (3, 3)
plt.rcParams['font.size'] = 12

sc.pl.umap(adata_concat, color=['batch_name', 'Ground Truth', 'mclust'], ncols=3,
           wspace=0.5, show=False)
plt.savefig(out_dir + expriment_name + '_umap.png', dpi=300)

In [ ]:
Batch_list = []
for section_id in section_ids:
    Batch_list.append(adata_concat[adata_concat.obs['batch_name'] == section_id])

import matplotlib.pyplot as plt
spot_size = 200
title_size = 12
ARI_list = []
for bb in range(len(Batch_list)):
    ARI_list.append(round(ari_score(Batch_list[bb].obs['Ground Truth'], Batch_list[bb].obs['mclust']), 2))

fig, ax = plt.subplots(1, len(Batch_list), figsize=(10, 5), gridspec_kw={'wspace': 0.05, 'hspace': 0.1})
_sc_0 = sc.pl.spatial(Batch_list[0], img_key=None, color=['mclust'], title=[''],
                      legend_loc=None, legend_fontsize=12, show=False, ax=ax[0], frameon=False,
                      spot_size=spot_size)
_sc_0[0].set_title("ARI=" + str(ARI_list[0]), size=title_size)
_sc_1 = sc.pl.spatial(Batch_list[1], img_key=None, color=['mclust'], title=[''],
                      legend_loc=None, legend_fontsize=12, show=False, ax=ax[1], frameon=False,
                      spot_size=spot_size)
_sc_1[0].set_title("ARI=" + str(ARI_list[1]), size=title_size)
# _sc_2 = sc.pl.spatial(Batch_list[2], img_key=None, color=['mclust'], title=[''],
#                       legend_loc=None, legend_fontsize=12, show=False, ax=ax[2], frameon=False,
#                       spot_size=spot_size)
# _sc_2[0].set_title("ARI=" + str(ARI_list[2]), size=title_size)
# _sc_3 = sc.pl.spatial(Batch_list[3], img_key=None, color=['mclust'], title=[''],
#                       legend_fontsize=12, show=False, ax=ax[3], frameon=False,
#                       spot_size=spot_size)
# _sc_3[0].set_title("ARI=" + str(ARI_list[3]), size=title_size)

plt.savefig(out_dir + expriment_name + '_ARI.png', dpi=300)
plt.show()

# save h5ad

In [ ]:
print(type(adata_concat))
print(adata_concat.obsm)
print(adata_concat)

#  tuple 不能保存为h5ad
adata_concat.uns['edgeList'] = list(adata_concat.uns['edgeList'])

print(adata_concat.isbacked)
adata_concat.filename = out_dir + expriment_name + '.h5ad'
print(adata_concat.isbacked)